---
# **RUN on dataset**
---

In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.analysis.stats import StatAnalysis
from pyVHR.plot.visualize import *
import os
import pandas as pd
import plotly.express as px
from numpy import round as r
import plotly.io as pio

# set the right renderer for plotly: 'jupyterlab', 'colab', etc
pio.renderers.default = 'vscode'

# main functions 

def ANALYSIS(dataset, approach, metrics=['MAE'], j_data=False, writeFile=False):
  """Print boxplot analysis"""

  cd = os.getcwd() # current dir
  if not j_data:
    print('## Dataset: ' + dataset)
    print('   Approach: ' + approach)
    path = path_results + dataset + "_" + approach + ".h5"
    print('   h5      :', path)
    cfg = path_results + dataset + "_" + approach + ".cfg"
    print('   cfg     :', cfg, end='\n\n')
  else:
    print('## Path dir: ' + dataset)
    print('   Approach: ' + approach)
    path = path_results + dataset 
    print('   path  = ', path)

  filenameH5 = os.path.join(cd, path)
  
  # boxplot
  for m in metrics:
    st = StatAnalysis(filenameH5, join_data=j_data)
    if m=='MAE': 
      scale = 'log' 
    else: 
      scale = 'linear'
    fig = st.displayBoxPlot(metric=m, scale=scale)
    fig.show()
    if writeFile:
      filenamePNG = path_results + dataset + "_" + approach + "_" + m + "_MAE.png"
      fig.write_image(filenamePNG)  

def STATS(dataset, approach='clustering'):
  """Do stats analysis"""
  
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + approach + ".h5"
  print(os.path.join(cd, path))
  df = pd.read_hdf(os.path.join(cd, path))

  # compute errors
  MAE_mean, MAE_std, MAE_median =    r(df['MAE'].mean()[0],2),  r(df['MAE'].apply(lambda x: x[0]).std(),2),  r(df['MAE'].apply(lambda x: x[0]).median(),2)
  RMSE_mean, RMSE_std, RMSE_median = r(df['RMSE'].mean()[0],2), r(df['RMSE'].apply(lambda x: x[0]).std(),2), r(df['RMSE'].apply(lambda x: x[0]).median(),2)
  PCC_mean, PCC_std, PCC_median =    r(df['PCC'].mean()[0],2),  r(df['PCC'].apply(lambda x: x[0]).std(),2),  r(df['PCC'].apply(lambda x: x[0]).median(),2)
  CCC_mean, CCC_std, CCC_median =    r(df['CCC'].mean()[0],2),  r(df['CCC'].apply(lambda x: x[0]).std(),2),  r(df['CCC'].apply(lambda x: x[0]).median(),2)
  SNR_mean, SNR_std, SNR_median =    r(df['SNR'].mean()[0],2),  r(df['SNR'].apply(lambda x: x[0]).std(),2),  r(df['SNR'].apply(lambda x: x[0]).median(),2)
  MAX_mean, MAX_std, MAX_median =    r(df['MAX'].mean()[0],2),  r(df['MAX'].apply(lambda x: x[0]).std(),2),  r(df['MAX'].apply(lambda x: x[0]).median(),2)

 # print table 
  table = [[MAE_mean, MAE_std, MAE_median], [RMSE_mean, RMSE_std, RMSE_median], [PCC_mean, PCC_std, PCC_median],
           [CCC_mean, CCC_std, CCC_median], [SNR_mean, SNR_std, SNR_median], [MAX_mean, MAX_std, MAX_median]]
  stats = pd.DataFrame(table, columns = [ 'MEANS', 'STD', 'MEDIANS'], index=['MAE', 'RMSE', 'PCC', 'CCC', 'SNR', 'MAX'])
  print('\n** Stats table **')
  print(stats)


def RUN(dataset, method, filenameH5=None):
  """Run methods on a dataset"""

  # set paths
  print('## Dataset: ' + dataset)
  print('Using...')
  cd = os.getcwd() # current dir
  if filenameH5 is None:
    filenameH5 = path_results + 'h5/' + dataset + "_" + method + ".h5"
  print('   filename h5: ', filenameH5)
  cfg = path_results  + 'cfg/'+ dataset + "_" + method + ".cfg"
  print('   cfg        :', cfg)
  print('\n')

  # pipeline
  pl = Pipeline()
  res = pl.run_on_dataset(os.path.join(cd, cfg), verb=1)
  res.saveResults(filenameH5)
  print('Written file: ' + filenameH5 + '\n\n')

def DETAILS(dataset, method, metric='MAE'):
  """Show performance details on each video of the dataset"""

  # set paths
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + method + ".h5"
  print('## Dataset: ' + dataset)
  print('   filename h5: ', path)
  df = pd.read_hdf(os.path.join(cd, path))
  datasetname = df['dataset'][0].upper()
  methods = set(list(df['method']))

  # loop on methods
  for m in methods:
    print('   method     :', m)
    vals = df[df['method'] == m][metric]
    fvideos = df[df['method'] == m]['videoFilename']
    x = []
    y = []
    for i in vals.index:
      n = fvideos[i]
      # n = n[n.rfind('/', 0, n.rfind('/'))+1:n.rfind('/')]
      n = n[n.rfind('\\', 0, n.rfind('\\'))+1:n.rfind('\\')]
      x.append(n)
      y.append(vals[i][0])
    fig = px.bar(x=x, y=y, title='dataset: ' + datasetname + ' method: ' + m)
    fig.update_xaxes(type='category')
    fig.show()

This notebook shows the complete **pipeline** that takes a **dataset** as input and computes an **estimation** of **BPMs** using a prespecified set of **rPPG** **methods**.

Below are the possible choices for the methods.
* **Methods**: `cpu_CHROM`, `cupy_CHROM`, `torch_CHROM`, `cpu_LGI`, `cpu_POS`, `cupy_POS`, `cpu_PBV`, `cpu_PCA`, `cpu_GREEN`, `cpu_OMIT`, `cpu_ICA`, `cpu_SSR`

Below are the possible choices for the datasets.
* **Datasets**: `PURE`, `UBFC1`, `UBFC2`, `ECG_Fitness_01-1`, `ECG_Fitness_01-2`... `ECG_Fitness_06-1`, `ECG_Fitness_06-2`

# Runs

In [16]:
# run on dataset

path_results = "../results/" # general path for cfg

dataset = 'LGI_PPGI'
filenameH5 = path_results + 'test/' + dataset + "_" + 'holistic' + ".h5"
RUN(dataset, 'holistic', filenameH5=filenameH5)
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')

## Dataset: LGI_PPGI
Using...
   filename h5:  ../results/test/LGI_PPGI_holistic.h5
   cfg        : ../results/cfg/LGI_PPGI_holistic.cfg


** Run the test with the following config:
      dataset: LGI_PPGI
      methods: ['CHROM', 'POS', 'LGI', 'ICA', 'PCA']
Dataset :  lgi_ppgi D:/datasets_rppg/lgi_ppgi D:/datasets_rppg/lgi_ppgi
Number of videos in folder :  24
Number of videos in index :  0
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3080
# device number  1 :  NVIDIA GeForce RTX 3080
 -  cuda device: True
 -  skin extractor: convexhull
 -  ROI approach: holistic
Why is it resetting video index ? :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

## videoID: 0
D:/datasets_rppg/lgi_ppgi\alex\alex_gym\cv_camera_sensor_stream_handler.avi
## method: CHROM (holistic)
  pre-filter: BPFILTER
  post-filter: BPFILTER

    * Errors: RMSE = 39.42, MAE = 27.38, MAX = 134.08, PCC = 0.26, CCC = 0.19, SNR = -4.24
## method: POS (holistic)

    *

c:\Users\erolland\.conda\envs\project\lib\site-packages\tables\path.py:137: NaturalNameWarning:

object name is not a valid Python identifier: 'self.dataFrame'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

c:\users\erolland\documents\pyvhr\pyVHR\analysis\pipeline.py:1095: PerformanceWarning:


your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['method', 'dataset', 'sigFilename', 'videoFilename', 'RMSE', 'MAE',
       'PCC', 'CCC', 'SNR', 'MAX', 'MAD', 'bpmGT', 'bpmES', 'bpmES_mad',
       'timeGT', 'timeES', 'TIME_REQUIREMENT'],
      dtype='object')]




# Analysis

In [3]:
## -- Holistic

# path_results = "../results/h5/" # general path for cfg# general path for data
path_results = "../results/test/" # general path for h5

dataset = 'LGI_PPGI'
ANALYSIS(dataset, 'holistic', metrics=['RMSE'])
STATS(dataset, approach='holistic')

## Dataset: LGI_PPGI
   Approach: holistic
   h5      : ../results/test/LGI_PPGI_holistic.h5
   cfg     : ../results/test/LGI_PPGI_holistic.cfg



c:\Users\erolland\Documents\pyVHR\notebooks\../results/test/LGI_PPGI_holistic.h5

** Stats table **
      MEANS    STD  MEDIANS
MAE   11.56  10.30     8.32
RMSE  18.55  14.30    15.43
PCC    0.23   0.37     0.17
CCC    0.20   0.33     0.06
SNR    1.78   4.29     1.15
MAX   62.47  41.59    50.89


In [4]:
## -- Holistic

# path_results = "../results/h5/" # general path for cfg# general path for data
path_results = "../results/test/" # general path for h5

dataset = 'LGI_PPGI'
ANALYSIS(dataset, 'holistic', metrics=['PCC'])
STATS(dataset, approach='holistic')

## Dataset: LGI_PPGI
   Approach: holistic
   h5      : ../results/test/LGI_PPGI_holistic.h5
   cfg     : ../results/test/LGI_PPGI_holistic.cfg



c:\Users\erolland\Documents\pyVHR\notebooks\../results/test/LGI_PPGI_holistic.h5

** Stats table **
      MEANS    STD  MEDIANS
MAE   11.56  10.30     8.32
RMSE  18.55  14.30    15.43
PCC    0.23   0.37     0.17
CCC    0.20   0.33     0.06
SNR    1.78   4.29     1.15
MAX   62.47  41.59    50.89


In [27]:
## -- Median
path_results = "../results/h5/" # general path for cfg# general path for data
ANALYSIS(dataset, 'median', metrics=['PCC'])
STATS(dataset, approach='median')

## Dataset: LGI_PPGI
   Approach: median
   h5      : ../results/h5/LGI_PPGI_median.h5
   cfg     : ../results/h5/LGI_PPGI_median.cfg



c:\Users\erolland\Documents\pyVHR\notebooks\../results/h5/LGI_PPGI_median.h5

** Stats table **
      MEANS    STD  MEDIANS
MAE    7.59   8.39     4.94
RMSE  10.30  10.05     6.29
PCC    0.35   0.42     0.34
CCC    0.31   0.36     0.16
SNR    0.26   5.14    -1.13
MAX   30.50  26.56    19.54


In [28]:
## -- Clustering
path_results = "../results/h5/" # general path for cfg# general path for data
ANALYSIS(dataset, 'clustering', metrics=['PCC'])
STATS(dataset, approach='clustering')

## Dataset: LGI_PPGI
   Approach: clustering
   h5      : ../results/h5/LGI_PPGI_clustering.h5
   cfg     : ../results/h5/LGI_PPGI_clustering.cfg



c:\Users\erolland\Documents\pyVHR\notebooks\../results/h5/LGI_PPGI_clustering.h5

** Stats table **
      MEANS    STD  MEDIANS
MAE    7.42   8.84     4.32
RMSE   9.98  10.72     5.64
PCC    0.35   0.44     0.40
CCC    0.33   0.36     0.26
SNR    0.26   5.14    -1.13
MAX   25.48  23.76    17.64


In [29]:
## -- Deep 
# path_results = "../results/h5/" # general path for cfg# general path for data
ANALYSIS(dataset, 'deep', metrics=['PCC'])
STATS(dataset, approach='deep')

## Dataset: LGI_PPGI
   Approach: deep
   h5      : ../results/h5/LGI_PPGI_deep.h5
   cfg     : ../results/h5/LGI_PPGI_deep.cfg



c:\Users\erolland\Documents\pyVHR\notebooks\../results/h5/LGI_PPGI_deep.h5

** Stats table **
      MEANS    STD  MEDIANS
MAE   18.17  13.66    16.90
RMSE  23.21  15.27    22.91
PCC    0.05   0.36    -0.02
CCC    0.08   0.23    -0.00
SNR   -0.94   6.18    -0.78
MAX   55.13  38.23    49.95


# Details

In [31]:
df = pd.read_hdf('../results/test/LGI_PPGI_holistic.h5')

In [26]:
# holistic
path_results = "../results/test/" # general path for cfg# general path for data
method = 'holistic'
DETAILS(dataset, method, metric='PCC')

## Dataset: LGI_PPGI
   filename h5:  ../results/test/LGI_PPGI_holistic.h5
   method     : POS


   method     : ICA


   method     : PCA


   method     : LGI


   method     : CHROM


In [10]:
# median

method = 'median'
DETAILS(dataset, method, metric='PCC')

## Dataset: LGI_PPGI
   filename h5:  ../results/h5/LGI_PPGI_median.h5
   method     : PCA


   method     : CHROM


   method     : LGI


   method     : ICA


   method     : POS


In [11]:
# clustering

method = 'clustering'
DETAILS(dataset, method, metric='PCC')

## Dataset: LGI_PPGI
   filename h5:  ../results/h5/LGI_PPGI_clustering.h5
   method     : PCA


   method     : CHROM


   method     : LGI


   method     : ICA


   method     : POS


# Multidataset

In [326]:
# multi dataset
#path = path_results + 'h5_collection'
#st = StatAnalysis(path, join_data=True)
#fig = st.displayBoxPlot(metric='PCC')
#fig.show(renderer = "colab")
